In [37]:
%pylab inline
import re
import math
import string
from collections import Counter
from __future__ import division

Populating the interactive namespace from numpy and matplotlib


In [38]:
lines = open('spell-errors.txt').read().splitlines() #read all the words from big.txt

In [39]:
dicti = {}

In [40]:
def edit_distance(s1, s2):
   m=len(s1)+1
   n=len(s2)+1

   tbl = {}
   for i in range(m): tbl[i,0]=i
   for j in range(n): tbl[0,j]=j
   for i in range(1, m):
       for j in range(1, n):
           cost = 0 if s1[i-1] == s2[j-1] else 1
           tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

   return tbl[i,j]

print(edit_distance("ordinary", "ordenary"))

1


In [41]:
import csv
w = csv.writer(open("dicti.csv", "w"))
for key, val in dicti.items():
    w.writerow([key, val])

In [42]:
TEXT = open('big.txt').read() #read all the words from big.txt
len(TEXT)

6488666

In [43]:
#we use re package to filter out unnecessary characters, tokenize the words and convert it to lower case
def tokens(text):
    return re.findall('[a-z]+', text.lower()) 

In [44]:
WORDS = tokens(TEXT)
len(WORDS)

1105285

In [45]:
COUNTS = Counter(WORDS)
print(COUNTS.most_common(10))
print(COUNTS.get('and'))

[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]
38313


In [46]:
for line in lines:
    word = line.split(':')
    cword = word[0].strip()
    wwords = word[1].split(',')
    for wword in wwords:
        if wword.strip() in dicti:
            costold = edit_distance(wword.strip(), dicti.get(wword.strip()))
            costnew = edit_distance(wword.strip(), cword)
            if costnew < costold:
                dicti[wword.strip()] = cword
            elif costold == costnew:
                cnt1 = COUNTS.get(cword)
                cnt2 = COUNTS.get(dicti.get(wword.strip()))
                if cnt1 and cnt2:
                    if cnt1 > cnt2:
                        dicti[wword.strip()] = cword
        else:
            dicti[wword.strip()] = cword

In [47]:
len(dicti)

37458

In [48]:
def correct(word):
    #Generating all the words with edit distance of 0, 1 & 2
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

In [49]:
def known(words):
    #Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    return {word}

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [50]:
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [51]:
def correct_text(text):
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [52]:
import pandas as pd

DF = pd.read_csv('test.csv')
len(DF)

504

In [53]:
DF.head()

,ID,WRONG
0,0,contenpted
1,1,begining
2,2,problam
3,3,dirven
4,4,exstacy


In [54]:
X = DF.iloc[:,1].values
print(len(X))

504


In [55]:
def tofind(word):
    answer = dicti.get(word.strip(),'hi')
    if answer == 'hi':
        answer = correct(word)
    return answer

In [56]:
Y = list(map(tofind, X))

In [57]:
Y

['contented',
 'looking',
 'problem',
 'driven',
 'ecstasy',
 'juice',
 'locally',
 'compare',
 'pronunciation',
 'transportability',
 'miniscule',
 'independent',
 'arranged',
 'poetry',
 'level',
 'basically',
 'triangular',
 'unexpected',
 'standardizing',
 'variable',
 'further',
 'monitoring',
 'biscuits',
 'available',
 'separate',
 'necessary',
 'definition',
 'receipt',
 'remind',
 'initials',
 'magnificent',
 'aunt',
 'initial',
 'the',
 'experiences',
 'built',
 'totally',
 'understand',
 'southern',
 'definitely',
 'visited',
 'voluntary',
 'meant',
 'receive',
 'sources',
 'whether',
 'useful',
 'literature',
 'valuable',
 'delicate',
 'clerical',
 'splendid',
 'between',
 'completely',
 'account',
 'cemetery',
 'special',
 'latest',
 'perhaps',
 'remember',
 'chapter',
 'cake',
 'various',
 'february',
 'pretend',
 'choosing',
 'rota',
 'particular',
 'awful',
 'arrangement',
 'challenges',
 'laugh',
 'often',
 'someone',
 'personnel',
 'unique',
 'diagrammatically',
 'des

In [58]:
np.savetxt(r'C:\Users\Administrator\IDS\test folder\npvabmix.txt', Y, fmt='%s')